Please read explanation in the RS first.
The common things are explained in the RS file.

In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [ ]:
import aiida
aiida.load_profile()


In [ ]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [ ]:
!pwd

In [ ]:
import os
import numpy as np

from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

# from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

# PandasFrameData = DataFactory('cryspy.dataframe')
PandasFrameData = DataFactory('dataframe.frame')

ConfigparserData = DataFactory('cryspy.configparser')
StructurecollectionData = DataFactory('cryspy.structurecollection')
LAQAData = DataFactory('cryspy.laqa_data')
LAQAidData = DataFactory('cryspy.laqa_id_data')


In [ ]:
pwd = os.getcwd()
cryspy_in = SinglefileData(os.path.join(pwd, 'cryspy.in0'))
inputs = {'cryspy_in': cryspy_in}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


In [ ]:
# workflow結果表示
initial_result


In EA, detail_data are used along with id_data. 

id_data contains queue and running information, but they aren't used in AiiDA.


In [ ]:
initial_structures_node = initial_result["initial_structures"]
rslt_data_node = initial_result["rslt_data"]
id_data_node = initial_result["id_data"]
detail_data_node = initial_result["detail_data"]
stat_node = initial_result["stat"]
cryspy_in_node = initial_result["cryspy_in"]


In [ ]:
import shutil
# cryspy.inは実行中に書き換えられる。このpackageでは読まれないはずだが念のためにコピーする。
shutil.copy("cryspy.in0", "cryspy.in")


# 

# select structures to optimize

In [ ]:
id_data_node.laqa_id_data


In [ ]:
inputs = {"id_data": id_data_node, "initial_structures":initial_structures_node }

select_structure_to_run_WorkChain = WorkflowFactory('cryspy.select_structures')
select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["selected_structures"]


In [ ]:
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["selected_structures"]
id_node = select_structure_result["id_data"]

In [ ]:
# lammps code setup
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke" , 
                         "pxx", "pyy", "pzz", "pxy", "pxz", "pyz"],
    'thermo_keywords': ["pxx", "pyy", "pzz", "pxy", "pxz", "pyz"],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 5,
        'max_evaluations': 10000
        }
})

In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    

In [ ]:
optimize_result


In [ ]:
# outputs. results
print(optimize_result["results"].get_dict())


In [ ]:
optimize_result['step_data'].step_data[0]

In [ ]:
# final structures
optimized_structures_node = optimize_result["final_structures"]
print(optimized_structures_node.structurecollection.keys())
print("---")
print(optimized_structures_node.structurecollection[0])


# summary

In [ ]:
optimize_result["results"].get_dict()

In [ ]:
# optの対称性の登録
from aiida_cryspy.common import generate_rlst

rslt_data_node = generate_rlst(initial_structures_node, optimized_structures_node, 
                              optimize_result["results"], 
                              id_node, rslt_data_node, stat_node)

# rslt for new EA
rslt_data_node.df


Values must be set in  
<pre>
            (self.tot_step_select, self.laqa_step,
             self.laqa_struc, self.laqa_energy,
             self.laqa_bias, self.laqa_score) = pkl_data.load_laqa_data()
</pre>
from rslt_data and step_data.

In [ ]:
step_data_node = optimize_result['step_data']
step_data_node


In [ ]:
cryspy_in_node.rin.nselect_laqa


In [ ]:
from CrySPY.LAQA.calc_score import calc_laqa_bias
import pandas as pd
from aiida.engine import WorkChain


RinData = DataFactory('cryspy.rin_data')
LAQAStepData = DataFactory('cryspy.laqa_step_data')

def _calculate_laqa_scores(step_data_node, wf_laqa = 1.0, ws_laqa = 1.0):
    """calculate LAQA scores.
    
    calculate -energy + wf_laqa* force_bias + ws_lawa * stress.
    
    Args:
        step_data_node (LAQAStepData): history data of energy, force and stresses.
        wf_laqa (float): force factor.
        ws_laqa (float): stress factor.
    """
    step_data = step_data_node.step_data
    lines = []
    for key, each_step_data in step_data.items():
        wf_laqa_bias = calc_laqa_bias(each_step_data["force"], wf_laqa)
        ws_laqa_bias = np.linalg.norm(each_step_data["stress"][-1])* ws_laqa
        energy = each_step_data["energy"][-1]
        nstep = len(each_step_data["energy"])
        laqa_score = -energy + wf_laqa_bias+ ws_laqa_bias
        lines.append([key, energy, wf_laqa_bias, ws_laqa_bias, nstep, laqa_score])
    columns = ['ID','energy', 'wf_laqa_bias', 'ws_laqa_bias', 'nstep', 'score']
    laqa_df = pd.DataFrame(lines,  columns=columns)
    return laqa_df

def _extract_laqa_score(laqa_df):
    IDs = laqa_df["ID"].astype(int).values.tolist()
    scores = laqa_df["score"].astype(float).values.tolist()
    laqa_score = {}
    for ID,score in zip(IDs,scores):
        ID = int(ID)
        laqa_score.update({ID: score})

    return laqa_score

def laqa_next_selection(laqa_score, nselect_laqa):
    id_queueing = []
    for k, v in sorted(laqa_score.items(), key=lambda x: -x[1]):
        print(k,v)
        #if v == -float('int'):
        #    break
        id_queueing.append(k)
        if len(id_queueing) >= nselect_laqa:
            break
    return id_queueing

class generatte_laqa_data_WorkChain(WorkChain):

    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('cryspy_in', valid_type=RinData, help='parsed cryspy_in content.')
        spec.input('step_data', valid_type=LAQAStepData, help='step data.')
        spec.outline(cls.generate_laqa_data)
        spec.output('laqa_data', valid_type=LAQAData)
        spec.output('laqa_summary', valid_type=PandasFrameData)
    
    def generate_laqa_data(self):
        
        rin_node = self.inputs.cryspy_in
        step_data_node = self.inputs.step_data
        
        rin = rin_node.rin
        laqa_df = _calculate_laqa_scores(step_data_node, rin.wf_laqa, rin.ws_laqa)
        df_node = PandasFrameData(df=laqa_df)

        laqa_score = _extract_laqa_score(laqa_df)

        values = (step_data_node.step_data,laqa_score)
        laqa_data_node = LAQAData(values)
        laqa_data_node.store()
        df_node = PandasFrameData(df=laqa_df)
        df_node.store()

        self.out('laqa_data', laqa_data_node)
        self.out('laqa_summary', df_node)

inputs = {'cryspy_in': cryspy_in_node, 'step_data': step_data_node} 
generate_result = run(generatte_laqa_data_WorkChain, **inputs)

In [ ]:
generate_result["laqa_summary"].df

In [ ]:
generate_result["laqa_data"].laqa_data

In [ ]:
step_data_node.step_data.keys()

# next_sg

It generates new candidates by LAQA.


In [ ]:
from aiida.engine import WorkChain, calcfunction
from aiida.plugins import DataFactory
from CrySPY.BO.select_descriptor import select_descriptor
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.LAQA.calc_score import calc_laqa_bias

PandasFrameData = DataFactory('dataframe.frame')
StructurecollectionData = DataFactory('cryspy.structurecollection')

EAData = DataFactory('cryspy.ea_data')
BOData = DataFactory('cryspy.bo_data')
LAQAData = DataFactory('cryspy.laqa_data')

EAidData = DataFactory('cryspy.ea_id_data')
BOidData = DataFactory('cryspy.bo_id_data')
LAQAidData = DataFactory('cryspy.laqa_id_data')

LAQAStepData = DataFactory('cryspy.laqa_step_data')


ConfigparserData = DataFactory('cryspy.configparser')
RinData = DataFactory('cryspy.rin_data')


def _update_bo_data(rin_node, bo_data_node, rslt_data_node, optimized_structures_node):
    """
    update detail_data_node for the optimize structures.

    from ctrl_collect_bo.
    """
    rin = rin_node.rin
    bo_data = bo_data_node.bo_data
    opt_dscrpt_data = {}
    for ID in rslt_data_node.df["Opt"].index:
        status = rslt_data_node.df["Opt"].loc[ID]
        # Must be 'status' used?
        opt_struc = optimized_structures_node.structurecollection[ID]
        tmp_dscrpt = select_descriptor(rin, {ID: opt_struc})
        opt_dscrpt_data.update(tmp_dscrpt)
    opt_dscrpt_data.keys()
    detail_data_node = BOData((bo_data[0], opt_dscrpt_data, bo_data[3], bo_data[3], bo_data[4]))
    return detail_data_node


def _calculate_laqa_scores(step_data, wf_laqa=1.0, ws_laqa=1.0):
    """calculate LAQA scores.

    calculate -energy + wf_laqa* force_bias + ws_lawa * stress.

    Args:
        step_data (dict): history data of energy, force and stresses.
        wf_laqa (float): force factor.
        ws_laqa (float): stress factor.
    """
    lines = []
    for key, each_step_data in step_data.items():
        wf_laqa_bias = calc_laqa_bias(each_step_data["force"], wf_laqa)
        ws_laqa_bias = np.linalg.norm(each_step_data["stress"][-1]) * ws_laqa
        energy = each_step_data["energy"][-1]
        nstep = len(each_step_data["energy"])
        laqa_score = -energy + wf_laqa_bias + ws_laqa_bias
        lines.append([key, energy, wf_laqa_bias, ws_laqa_bias, nstep, laqa_score])
    columns = ['ID', 'energy', 'wf_laqa_bias', 'ws_laqa_bias', 'nstep', 'score']
    laqa_df = pd.DataFrame(lines,  columns=columns)
    return laqa_df


def _extract_laqa_score(laqa_df):
    IDs = laqa_df["ID"].astype(int).values.tolist()
    scores = laqa_df["score"].astype(float).values.tolist()
    laqa_score = {}
    for ID, score in zip(IDs, scores):
        ID = int(ID)
        laqa_score.update({ID: score})
    return laqa_score


def _update_step_data(laqa_data_node, step_data_node):
    step_data = laqa_data_node.laqa_data[0]
    new_step_data = step_data_node.step_data
    for key,value in new_step_data.items():
        step_data.update({key: value})
    return step_data


def _generate_laqa_data(rin_node, laqa_data_node, step_data_node):
    rin = rin_node.rin
    updated_step_data = _update_step_data(laqa_data_node, step_data_node)
    laqa_df = _calculate_laqa_scores(updated_step_data, rin.wf_laqa, rin.ws_laqa)
    laqa_score = _extract_laqa_score(laqa_df)
    laqa_node = LAQAData((updated_step_data, laqa_score))
    df_node = PandasFrameData(df=laqa_df)
    return laqa_node, df_node


def _laqa_next_selection(laqa_data_node, nselect_laqa):
    laqa_score = laqa_data_node.laqa_data[1]
    id_queueing = []
    for k, v in sorted(laqa_score.items(), key=lambda x: -x[1]):
        print(k, v)
        # if v == -float('int'):
        #    break
        id_queueing.append(k)
        if len(id_queueing) >= nselect_laqa:
            break
    return id_queueing


class next_sg_WorkChain(WorkChain):
    """next_sg()

    Q. Why are the initial_strucutures as input  necessary?
    A. It is to add new structures to the initial_structures.

    """
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input("initial_structures", valid_type=StructurecollectionData, help='initial structures')
        spec.input("optimized_structures", valid_type=StructurecollectionData, help='optimized structures')
        spec.input("rslt_data", valid_type=PandasFrameData, help='summary data')
        spec.input("id_data", valid_type=(EAidData, BOidData, LAQAidData), help='id_data')
        spec.input('step_data', valid_type=LAQAStepData, required=False, help='optimization step data')
        spec.input("detail_data", valid_type=(EAData, BOData, LAQAData), help='detail_data')
        spec.input("stat", valid_type=ConfigparserData, help='cryspy_in content')
        spec.input('cryspy_in', valid_type=RinData, help='cryspy_in')

        spec.outline(
            cls.validate_inputdata,
            cls.call_next_sg
        )

        spec.output('id_data', valid_type=(EAidData, BOidData, LAQAidData))
        spec.output('detail_data', valid_type=(EAData, BOData, LAQAData))
        spec.output('rslt_data', valid_type=PandasFrameData)
        spec.output('initial_structures', valid_type=StructurecollectionData)
        spec.output("stat", valid_type=ConfigparserData)
        spec.output('cryspy_in', valid_type=RinData)

    def validate_inputdata(self):
        id_data = self.inputs.id_data
        detail_data = self.inputs.detail_data
        # both data must be consistent
        if isinstance(id_data, EAidData):
            algo = "EA"
        elif isinstance(id_data, BOidData):
            algo = "BO"
        elif isinstance(id_data, LAQAidData):
            algo = 'LAQA'
        else:
            raise TypeError(f'unknown type for id_data. type={type(id_data)}')

        if isinstance(detail_data, EAData):
            algo_data = "EA"
        elif isinstance(detail_data, BOData):
            algo_data = "BO"
        elif isinstance(detail_data, LAQAData):
            algo_data = "LAQA"
        else:
            raise TypeError(f'unknown type for detail_data. type={type(detail_data)}')

        if algo != algo_data:
            raise TypeError(f'type(id_data) != type(detail_data), type={type(id_data)}, type={type(detail_data)}')
        self.ctx.algo = algo

    def call_next_sg(self):

        initial_structures = self.inputs.initial_structures.structurecollection
        opt_struc = self.inputs.optimized_structures.structurecollection
        rslt_data = self.inputs.rslt_data.df
        algo = self.ctx.algo

        id_data = self.inputs.id_data
        if isinstance(id_data, EAidData):
            algo = 'EA'
            id_data = id_data.ea_id_data
        elif isinstance(id_data, BOidData):
            algo = "BO"
            id_data = id_data.bo_id_data
        elif isinstance(id_data, LAQAidData):
            algo = 'LAQA'
            id_data = id_data.laqa_id_data
        # the other types are added.
        else:
            raise TypeError(f'internal error: unknown type for id_data, type={type(id_data)}')

        if algo == "EA":
            detail_data = self.inputs.detail_data.ea_data
        # algo=="RS" doesn't come to this function.
        elif algo == "BO":
            detail_data_node = _update_bo_data(self.inputs.cryspy_in,
                                               self.inputs.detail_data,
                                               self.inputs.rslt_data,
                                               self.inputs.optimized_structures)
            detail_data = detail_data_node.bo_data
        elif algo == 'LAQA':
            step_data_node = self.inputs.step_data
            detail_data_node = self.inputs.detail_data
            detail_data_node, df_laqa_score = _generate_laqa_data(self.inputs.cryspy_in, detail_data_node, step_data_node)
            detail_data_node.store()
            # df_laqa_score.store()
        else:
            raise TypeError(f'internal error: unknown type for id_data, type={type(id_data)}')

        rin = self.inputs.cryspy_in.rin
        stat = self.inputs.stat.configparser

        if algo == "BO":
            # BO doesn't increase structures.
            jobs = Ctrl_job(rin, stat, initial_structures,
                            opt_struc,
                            rslt_data, id_data, detail_data
                            )
            rin, stat, _, id_data, detail_data, rslt_data = jobs.next_sg()
            id_data_node = BOidData(id_data)
            id_data_node.store()
            detail_data_node = BOData(detail_data)
            detail_data_node.store()
            struc_node = self.inputs.initial_structures  # the same node
        elif algo == "EA":
            jobs = Ctrl_job(rin, stat, initial_structures,
                            opt_struc,
                            rslt_data, id_data, detail_data
                            )
            rin, stat, init_struc_data, id_data, detail_data, rslt_data = jobs.next_sg()
            id_data_node = EAidData(id_data)
            id_data_node.store()
            detail_data_node = EAData(detail_data)
            detail_data_node.store()
            struc_node = StructurecollectionData(init_struc_data)
            struc_node.store()
        elif algo == 'LAQA':
            id_queueing = _laqa_next_selection(detail_data_node, rin.nselect_laqa)
            id_data_node = LAQAidData((id_queueing, [], []))
            id_data_node.store()
            struc_node = self.inputs.optimized_structures

        rslt_data_node = PandasFrameData(rslt_data)
        rslt_data_node.store()

        stat_node = ConfigparserData(stat)
        stat_node.store()

        rin_node = RinData(rin)
        rin_node.store()

        self.out("id_data", id_data_node)
        self.out("detail_data", detail_data_node)
        self.out("rslt_data", rslt_data_node)
        self.out('initial_structures', struc_node)
        self.out('stat', stat_node)
        self.out('cryspy_in', rin_node)


In [ ]:
inputs = {'initial_structures': initial_structures_node,
         'optimized_structures': optimized_structures_node,
          'rslt_data': rslt_data_node,
          'id_data': id_node,
          'detail_data': detail_data_node,
          'step_data': step_data_node, 
         'stat': stat_node,
         'cryspy_in': cryspy_in_node}
# next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)


In [ ]:
next_sg_results


In [ ]:
initial_structures_node = next_sg_results["initial_structures"]
cryspy_in_node = next_sg_results["cryspy_in"]
stat_node = next_sg_results["stat"]
id_node = next_sg_results["id_data"]
detail_data_node = next_sg_results["detail_data"]
rslt_data = next_sg_results["rslt_data"]


In [ ]:
rslt_data.df.sort_values(by="E_eV_atom")


In [ ]:
id_node.laqa_id_data, initial_structures_node.structurecollection

In [ ]:
detail_data_node.laqa_data[0].keys()

# next structure  optimization

In [ ]:
# The next step select IDs in id_node.ea_id_data[1] from initial_structures_node.structurecollection
print(initial_structures_node.structurecollection.keys())
print(id_node.laqa_id_data)


In [ ]:
inputs = {"id_data":  id_node, "initial_structures": initial_structures_node }
select_structure_result = run(select_structure_to_run_WorkChain, **inputs)


In [ ]:
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["selected_structures"]
print(work_path_node.get_dict().keys(),  structures_to_optimize_node.get_dict().keys())


In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)


In [ ]:
optimize_result

In [ ]:
print(optimize_result["results"].get_dict())

In [ ]:
# merge data to optimized_structures_node
print(optimized_structures_node.structurecollection.keys())
print(optimize_result["final_structures"].structurecollection.keys())

In [ ]:
from aiida_cryspy.common import merge_structurecollection
optimized_structures_node = merge_structurecollection(optimized_structures_node, 
                                                          optimize_result["final_structures"])
print("merged optimized_structures_node",
optimized_structures_node.structurecollection.keys())

In [ ]:
id_node = next_sg_results["id_data"]
initial_structures_node = next_sg_results["initial_structures"]
print(initial_structures_node.structurecollection.keys())
print(optimized_structures_node.structurecollection.keys())

In [ ]:
rslt_data_node = generate_rlst(initial_structures_node, optimized_structures_node, 
                              optimize_result["results"], 
                              id_node, rslt_data_node, stat_node)
rslt_data_node.df.sort_values(by="E_eV_atom")

# next_sg

In [ ]:
step_data_node = optimize_result['step_data']


In [ ]:
inputs = {'initial_structures': initial_structures_node,
         'optimized_structures': optimized_structures_node,
          'rslt_data': rslt_data_node,
          'id_data': id_data_node,
          'detail_data': detail_data_node,
          'step_data': step_data_node, 
         'stat': stat_node, 'cryspy_in': cryspy_in_node}

# next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)


In [ ]:
next_sg_results


In [ ]:
initial_structures_node = next_sg_results["initial_structures"]
cryspy_in_node = next_sg_results["cryspy_in"]
stat_node = next_sg_results["stat"]
id_node = next_sg_results["id_data"]
detail_data_node = next_sg_results["detail_data"]
rslt_data = next_sg_results["rslt_data"]


# structure optimization 

In [ ]:
inputs = {"id_data":  id_node, "initial_structures": initial_structures_node }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["selected_structures"]


In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

In [ ]:
optimize_result

In [ ]:
id_node = next_sg_results["id_data"]
all_initial_structures_node = next_sg_results["initial_structures"]
optimized_structures_node = merge_structurecollection(optimized_structures_node, 
                                                      optimize_result["final_structures"])
print("all the optimized structures")
print(optimized_structures_node.structurecollection.keys())


In [ ]:
rslt_data_node = generate_rlst(initial_structures_node, optimized_structures_node, 
                              optimize_result["results"], 
                              id_node, rslt_data_node, stat_node)
print(rslt_data_node.df.shape)
rslt_data_node.df.sort_values(by="E_eV_atom").head(50)

In [ ]:
!verdi node graph generate $rslt_data_node.pk
# The resulting image is too big to show in jupyter.